In [2]:
print ("Hello")

Hello


## Lesson 3 - Data Analysis ##
### Activity ###
To Analyze given CSV files
1. enrollments.csv -
1. daily_engagement.csv -
1. project_submissions.csv -

#### Step 1: Read CSV Files ####
We use python "csv"  or "unicodecsv" modules for reading csv file provided.